# Task 1: Kaggle Cats and Dogs Classification

# Part 2: Testing

In [40]:
import torch
import torchvision as tv
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim 

# fix random seed for reproducability
torch.manual_seed(1)

In [41]:
# Parameters
path_test='./cats_and_dogs_filtered/validation/'
batch_size_test = 100 
num_workers_test = 0 
size_compressed = [80,80]

## Load the Previous CNN Model

In [42]:
# before loading the model para, construct the CNN with the same structure at first
class CNN(nn.Module):
    """ 
    Create CNN model with convolutional net (for feature extraction) and 
    fully connected layers (for binary classification)
    """
    def __init__(self):
        super(CNN, self).__init__()
        # define the number of neurons in convolutional layers
        num_filters = [1, 64, 64, 128, 128]
        # define the number of neurons in fully connected layers
        num_neurons_fully = [256, 64]
        
        # compute the first convolution layer, some details see: [1]
        # [1]: https://pytorch.org/docs/stable/nn.html?highlight=conv2d#torch.nn.Conv2d
        # input.size([1,80,80])
        self.con1 = nn.Conv2d(in_channels = num_filters[0], 
                      out_channels = num_filters[1], 
                      kernel_size = 5, 
                      padding = 1,
                      bias = False)  # output.size([64, 77, 77])
        # compute a max pooling layer
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # output.size([64, 39, 39])
        # batch nornamlization
        self.bn1 = nn.BatchNorm2d(num_features=num_filters[1])
        
        self.con2= nn.Conv2d(in_channels = num_filters[1],
                      out_channels = num_filters[2], 
                      kernel_size = 3, 
                      padding = 1,
                      bias = False)  # output.size([64, 38, 38])
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # output.size([64, 19, 19])
        self.bn2 = nn.BatchNorm2d(num_features=num_filters[2])
        
        self.con3 = nn.Conv2d(in_channels = num_filters[2],
                      out_channels = num_filters[3], 
                      kernel_size = 3, 
                      padding = 1,
                      bias = False)  # output.size([128, 18, 18])
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # output.size([128, 9, 9])
        self.bn3 = nn.BatchNorm2d(num_features=num_filters[3])
        
        self.con4 = nn.Conv2d(in_channels = num_filters[3],
                      out_channels = num_filters[4], 
                      kernel_size = 3, 
                      padding = 1,
                      bias = False)  # output.size([128, 8, 8])
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)  # output.size([128, 4, 4])
        self.bn4 = nn.BatchNorm2d(num_features=num_filters[4])
        
        self.fc1 = nn.Linear(num_filters[4]*4*4, num_neurons_fully[0], bias=True)
        self.fc2 = nn.Linear(num_neurons_fully[0], num_neurons_fully[1], bias=True)
        self.fc3 = nn.Linear(num_neurons_fully[1], 2, bias=True)
    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.con1(x))))
        x = self.pool2(F.relu(self.bn2(self.con2(x))))
        x = self.pool3(F.relu(self.bn3(self.con3(x))))
        x = self.pool4(F.relu(self.bn4(self.con4(x))))
        x = x.view(-1, 128*4*4)  # flatten to 1D
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x
    
kaggle_model = CNN()
kaggle_model = torch.load('./kaggle_model_2.pkl')

## Data Preprocessing

$\bullet$ Load test data and convert them to specific format (including changes of size, grayscale, number of channels and datatype) 

In [43]:
test_transform = tv.transforms.Compose([
    tv.transforms.Grayscale(num_output_channels=1), # reduce the channel Nr. to one
    tv.transforms.Resize(size_compressed),
    tv.transforms.ToTensor()
])
test_data = tv.datasets.ImageFolder(root = path_test, transform = test_transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size_test,
                                           num_workers = num_workers_test, shuffle = True)
print(test_data.class_to_idx) # return: dict with items (class_name, class_index)

{'cats': 0, 'dogs': 1}


## Test CNN Model

In [44]:
errors_test = 0
for batch_idx, (inputs,labels) in enumerate(test_loader):
    outputs = kaggle_model(inputs)
    # error rate
    predicted = torch.max(outputs,dim=1)
    errors_test += sum(predicted[1] != labels)
print('Error rate on test set:', round(100.0* errors_test.numpy() / len(test_loader.dataset),2), '%')

Error rate on test set: 22.6 %
